In [876]:
import requests
from tqdm.auto import tqdm
import time #для цикла, чтобы сделать паузу между обращениями к api hh
import joblib # для сохранкения дампа в файл
import math # для окпругления в большую сторону
import pandas as pd
from datetime import datetime # для работы с дата время
import re
import plotly.express as px
#для переворота словаря
from operator import itemgetter
import plotly as py
import plotly.graph_objs as go
from plotly.subplots import make_subplots

import nltk #стоп слова
import ssl #стоп слова
from nltk.corpus import stopwords
import pymorphy2 #для лемматизации
#для прогнозов sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.metrics import f1_score
RANDOM_STATE=123

# import numpy as np
# import os

# #для переворота словаря
# from operator import itemgetter


# Запрос вакансий с hh.ru

In [2]:
#определим список городом где есть плошадки МЦТП УП АРМ (мое подразделение) и для сравнения добавим туда Москву и Питер
cities = ['Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Нижний Новгород',  'Самара', 'Тула']

In [29]:
#Выясним номера регионов
areas = []
for city in cities:
    url = f'https://api.hh.ru/suggests/area_leaves?text={city}'
    res = requests.get(url)
    for item in res.json()['items']:
        if item['text'] in cities:
            areas.append(item)

In [31]:
areas

[{'id': '1', 'text': 'Москва', 'url': 'https://api.hh.ru/areas/1'},
 {'id': '2', 'text': 'Санкт-Петербург', 'url': 'https://api.hh.ru/areas/2'},
 {'id': '4', 'text': 'Новосибирск', 'url': 'https://api.hh.ru/areas/4'},
 {'id': '3', 'text': 'Екатеринбург', 'url': 'https://api.hh.ru/areas/3'},
 {'id': '66', 'text': 'Нижний Новгород', 'url': 'https://api.hh.ru/areas/66'},
 {'id': '78', 'text': 'Самара', 'url': 'https://api.hh.ru/areas/78'},
 {'id': '92', 'text': 'Тула', 'url': 'https://api.hh.ru/areas/92'}]

In [68]:
%%time
#Сохраним все номера id вакансий
page = 1
num_per_page = 100
all_vacancy_ids=[]
for region in tqdm(range(len(areas))):
    reg_id = areas[region]['id']
    time.sleep(60)
    #print(reg_id)
    url = f'https://api.hh.ru/vacancies?&page={page}&per_page={num_per_page}&area={reg_id}'
    for i in range(20):
        res = requests.get(url)
        vacancies = res.json()
        num_pages = vacancies.get('pages')
        vacancy_ids = [el.get('id') for el in vacancies.get('items')]
        all_vacancy_ids.extend(vacancy_ids)
        

  0%|          | 0/7 [00:00<?, ?it/s]

In [72]:
%%time
#сохраним номера вакансий в файл
#
joblib.dump(all_vacancy_ids,'all_vacancy_ids_dump')


['all_vacancy_ids_dump']

In [ ]:
#чтобы открыть =joblib.load('all_vacancy_ids_dump')

In [73]:
# проверяем число вакансий
len(all_vacancy_ids)

14000

In [111]:
%%time
#достаем все по вакансиям, подробно
vacancy_all_info = []
all_iteration = math.ceil((len(all_vacancy_ids))/50) #делим все вакансии на 50, и округляем в большую сторону.
for ii in range(all_iteration): #создал цикл по 50  вакнсий
    # Запрос исполняется Wall time: 5h 17min 16s
    x=50*ii
    ii+=1
    y=50*ii
    for i in range(x, y): #перебор вакансий с 0 по 49, с  50 по 99 и тд
        vac_id = all_vacancy_ids[i]
        url = f'https://api.hh.ru/vacancies/{vac_id}'
        res = requests.get(url)
        vacancy = res.json()
        vacancy_all_info.append(vacancy)
    #print(ii)
    time.sleep(60) #пауза в 60 секунд после каждого запроса на 50 вакансий, чтобы небыло капчи

Wall time: 5h 17min 16s


In [112]:
%%time
#сохраним номера вакансий в файл
#
joblib.dump(vacancy_all_info,'vacancy_all_info_dump')

Wall time: 14.2 s


['vacancy_all_info_dump']

In [568]:
#посмотрим на одну вакансию
vacancy_all_info[4000]

{'id': '78393684',
 'premium': False,
 'billing_type': {'id': 'standard', 'name': 'Стандарт'},
 'relations': [],
 'name': 'Сотрудник на производство картин по номерам',
 'insider_interview': None,
 'response_letter_required': False,
 'area': {'id': '4',
  'name': 'Новосибирск',
  'url': 'https://api.hh.ru/areas/4'},
 'salary': {'from': 36000, 'to': 40000, 'currency': 'RUR', 'gross': False},
 'type': {'id': 'open', 'name': 'Открытая'},
 'address': None,
 'allow_messages': True,
 'experience': {'id': 'noExperience', 'name': 'Нет опыта'},
 'schedule': {'id': 'fullDay', 'name': 'Полный день'},
 'employment': {'id': 'full', 'name': 'Полная занятость'},
 'department': None,
 'contacts': None,
 'description': '<p><em><strong>Мы ищем сотрудников на производство картин по номерам. Нам нужны трудолюбивые, ответственные люди. </strong></em></p> <p><em><strong>Эта работа для вас:</strong></em></p> <p><em><strong>если вы любите выполнять монотонную работу;</strong></em></p> <p><em><strong>если у ва

In [592]:
#Создаем DataFrame с нужными нам полями из всего массива данных
vac_info = pd.DataFrame(vacancy_all_info, columns = ['id', 'name', 'area','salary', 'experience', 'schedule', 
                                                     'employment', 'description', 'key_skills', 'professional_roles', 
                                                     'employer', 'published_at'])

In [926]:
#vac_info.info()

In [927]:
#Работаем с данными, очишаем, выбираем нужное и тд, в том числе через написанные функции

In [594]:
def name_of_property (slovar):
    """
    функция вытягивания конкретного свойства из поля
    """
    name_prop = slovar['name']
    return name_prop

In [595]:
# применем функцию и оставим нужные нам свойства
vac_info['area'] = vac_info['area'].apply(name_of_property)
vac_info['experience'] = vac_info['experience'].apply(name_of_property)
vac_info['schedule'] = vac_info['schedule'].apply(name_of_property)
vac_info['employment'] = vac_info['employment'].apply(name_of_property)
vac_info['employer'] = vac_info['employer'].apply(name_of_property)

In [596]:
def name_professional_roles (slovar):
    """
    функция вытягивания конкретного свойства из поля professional_roles
    """
    name_prop = slovar[0]['name']
    return name_prop

In [599]:
vac_info['professional_roles']=vac_info['professional_roles'].apply(name_professional_roles)

In [580]:
#функции для разбора поля salary на зарплату от и до 
def salary_from (slovar):
    """
    функция зарплата от
    """
    if type(slovar) ==dict:
        if type(slovar['from']) == int:
            salary_from = slovar['from']
            return salary_from

def salary_to (slovar):
    """
    функция зарплата до
    """
    if type(slovar) ==dict:
        if type(slovar['to']) == int:
            salary_to = slovar['to']
            return salary_to

In [613]:
#применим функции зарплат
vac_info['salary_from'] = vac_info['salary'].apply(salary_from)
vac_info['salary_to'] = vac_info['salary'].apply(salary_to)

In [614]:
# создаем функцию для конвертации в дату столбца 'published_at'
def convert_datetime(x):
    """
    переводим строку в дату 
    """
    if type(x)==str:
        new_x = datetime.strptime(x[:10], '%Y-%m-%d')
        return new_x
    else :
        new_x = None  # не знал как обойти когда вместо строки приходит что то иное
        return new_x

In [615]:
vac_info['published_at']=vac_info['published_at'].apply(convert_datetime)

In [616]:
#функция для преобразования требований в один список удобный нам
def vac_skill(list_skill):
    """
    Перебираем все существующие требования к навыкам в поле key_skills и записываем их в лист
    """
    if len(list_skill) != 0: #смотрим только существующие значения
        skill = []
        for i in range(len(list_skill)): 
            skill.append(list_skill[i]['name'])
        return skill

In [617]:
vac_info['key_skills']=vac_info['key_skills'].apply(vac_skill)

In [619]:
#выберем из vac_info.professional_roles.unique() цифровые специальности(смотрел что вообще дает HH и выбрал что связано с IT)
spisok = ['Тестировщик', 'Программист, разработчик','Специалист технической поддержки', 
          'Продуктовый аналитик', 'Руководитель группы разработки', 'Системный администратор', 'Менеджер продукта']

In [625]:
def find_vasc_roles (stroka):
    """
    функция поиска цифровых вакансий
    """
    sovpalo = 0
    for x in range(len(spisok)):
        if (spisok[x] == stroka) == True:
            sovpalo+=1
        #else: return 0
    if sovpalo > 0 : return 1
    else: return 0

In [628]:
# проставим flag = 1 там где цифровые вакансии
vac_info['flag'] = vac_info['professional_roles'].apply(find_vasc_roles)

In [630]:
#создаим копию датасета, и поработаем с данными
vac_info_new = vac_info.loc[vac_info['flag']==1].reset_index(drop=True).copy()

In [638]:
def vac_level(row):
    """
    функия опредления уровня вакансии из названия
    """
    if ('junior' in row)|('младший' in row)|('стажер' in row):
        return 'Junior'
    if 'middle' in row:
        return 'Middle'
    if ('senior' in row)|('старший' in row):
        return 'Senior'
    if ('chief' in row)|('head' in row)|('lead' in row)|('руководитель' in row):
        return 'Lead'

In [639]:
vac_info_new['vac_level']=vac_info_new['name'].apply(vac_level)

In [928]:
#Смотрим какие уровни есть
vac_info_new['vac_level'].unique()

array([None, 'Junior'], dtype=object)

In [ ]:
#к сожалению у нас уровень только новичок, исследования относительно уровня провести не возможно

In [647]:
#оценим данные
vac_info_new.isna().sum()

id                       0
name                     0
area                     0
salary                 460
experience               0
schedule                 0
employment               0
description              0
key_skills             340
professional_roles       0
employer                 0
published_at             0
salary_from            520
salary_to              820
flag                     0
vac_level             1040
dtype: int64

In [653]:
#построим график уровень зарплат от какой суммы (там где данные есть)
vac_info_new_salary_from = vac_info_new[vac_info_new['salary_from'].notnull()].copy()
table_for_plot = vac_info_new_salary_from
fig = px.box(table_for_plot, 
             x="area", 
             y="salary_from", 
             color = "area", 
             category_orders = {'area': table_for_plot['area'].value_counts().index}, 
             labels = {'area':'Регион', 'salary_from':'Зарплата от, руб.'})
fig.show()

In [654]:
#построим график уровень зарплат до (там где данные есть)
vac_info_new_salary_from = vac_info_new[vac_info_new['salary_to'].notnull()].copy()
table_for_plot = vac_info_new_salary_from
fig = px.box(table_for_plot, 
             x="area", 
             y="salary_to", 
             color = "area", 
             category_orders = {'area': table_for_plot['area'].value_counts().index}, 
             labels = {'area':'Регион', 'salary_to':'Зарплата от, руб.'})
fig.show()

In [ ]:
#видим из двух графиков как выровнялась сетка зарплат, вероятно из за ковида

In [ ]:
#проведем анализ данных

In [748]:
#определим топ 20 популярных навыков
skills_table = vac_info_new.copy()
skills_table = skills_table.filter(items = ["id", "name", "key_skills","salary_from","Level"])
skills_table_count = skills_table.explode("key_skills").groupby("key_skills").count()['id']
skills_table_count = pd.DataFrame(skills_table_count).rename(mapper = {'id':'count'}, axis = 1)
skills_table_count = skills_table_count.sort_values(by = 'count',ascending = False)
skills_table_count.head(20)

,count
key_skills,
SQL,220
JavaScript,200
Тестирование,160
Python,100
Atlassian Jira,80
HTML,80
TypeScript,80
Git,80
QA,80


In [730]:
#отобразим на графике
fig = px.bar(skills_table_count.head(20),labels = {'key_skills':'Навык', 'value':'Количество вакансий'})
fig.update_layout(showlegend=False)
fig.show()

In [929]:
#определим самые оплачиваемые навыки по среднему
skills_salary_table = skills_table.explode("key_skills").groupby("key_skills").mean().reset_index()
skills_salary_table = skills_salary_table.dropna(axis=0).sort_values(by='salary_from',ascending = False)
#skills_salary_table

In [778]:
def trim_string(x):
    """
    отрежем первые 25 символов навыка, для красоты отображения на графике
    """
    y=x[0:25]
    return y

In [780]:
skills_salary_table['key_skills']=skills_salary_table['key_skills'].apply(trim_string)

In [787]:
skills_salary_table = skills_salary_table.set_index(['key_skills'])

In [930]:
#skills_salary_table.head(20)

In [789]:
#отобразим на графике самые оплачиваемые навыки 
fig = px.bar(skills_salary_table.head(20),labels = {'skills':'Навык', 'value':'Средняя зарплата по вакансиям с навыком'})
fig.update_layout(showlegend=False)
fig.show()

In [909]:
# оценим количество вакансий по городам с навыками SQL и Python
area_table = vac_info_new.copy()
area_table = employer_table.filter(items = ["id", "employer","key_skills","area"])
area_table= employer_table.explode("key_skills")


In [910]:
area_table_count_sql = area_table[employer_table['key_skills']=='SQL'].groupby(['area']).count()['id']
area_table_count_sql = pd.DataFrame(area_table_count_sql).rename(mapper = {'id':'count'}, axis = 1)
area_table_count_sql  = area_table_count_sql.sort_values(by = 'count',ascending = False).reset_index()

In [911]:
area_table_count_Python = area_table[area_table['key_skills']=='Python'].groupby(['area']).count()['id']
area_table_count_Python = pd.DataFrame(area_table_count_Python).rename(mapper = {'id':'count'}, axis = 1)
area_table_count_Python  = area_table_count_Python.sort_values(by = 'count',ascending = False).reset_index()

In [931]:
# отобразим на графике
fig = make_subplots(rows=2, cols=1, shared_yaxes=False)

fig.add_trace(go.Bar(x = area_table_count_sql['area'], 
                     y = area_table_count_sql['count'],  
                     name='SQL'),
              1, 1)

fig.add_trace(go.Bar(x = area_table_count_Python['area'], 
                     y = area_table_count_Python['count'],  
                     name='Python'),
              2, 1)

    
fig.show()

In [ ]:
# хотел посмотреть по компаниям, но к сожалению как то так получилось что в основном по 20 вакансий от каждой компании
# и график просто ровный, не красивый. Так же пытался покрутить данные по навыкам в этих компаниях, их тоже по 20
# не интрестно. Ниже специально оставляю кусок вывода при разбивке, чтобы наглядно показать, что не интерестно

In [923]:
area_skills_table = vac_info_new.copy()
area_skills_table = area_skills_table.filter(items = ["id", "employer","key_skills","area"])
area_skills_table = area_skills_table.explode("key_skills")
area_skills_table_count = area_skills_table.groupby(['employer', 'key_skills']).count()['id']

In [925]:
area_skills_table_count

employer                     key_skills                 
2ГИС                         Docker                         20
                             Postman                        20
                             REST API                       20
                             Swagger                        20
                             Unix                           20
                                                            ..
Центр финансовых технологий  SQL                            20
                             Интеграционное тестирование    20
                             Теория тестирования            20
                             Тестирование                   20
                             Тестирование backend           20
Name: id, Length: 219, dtype: int64

In [ ]:
#оценим возможности работы на удаленке по городам

In [939]:
UD_table = vac_info_new.copy()
UD_table = UD_table.filter(items = ["id", "employer","schedule","area"])
#UD_table_count = UD_table.groupby(['area','schedule']).count()['id']

In [941]:
Office_table_count = UD_table[UD_table['schedule']=='Полный день'].groupby(['area']).count()['id']
Office_table_count = pd.DataFrame(Office_table_count).rename(mapper = {'id':'count'}, axis = 1)
Office_table_count  = Office_table_count.sort_values(by = 'count',ascending = False).reset_index()

In [943]:
Gib_table_count = UD_table[UD_table['schedule']=='Гибкий график'].groupby(['area']).count()['id']
Gib_table_count = pd.DataFrame(Gib_table_count).rename(mapper = {'id':'count'}, axis = 1)
Gib_table_count  = Gib_table_count.sort_values(by = 'count',ascending = False).reset_index()

In [946]:
UD_table_count = UD_table[UD_table['schedule']=='Удаленная работа'].groupby(['area']).count()['id']
UD_table_count = pd.DataFrame(UD_table_count ).rename(mapper = {'id':'count'}, axis = 1)
UD_table_count = UD_table_count .sort_values(by = 'count',ascending = False).reset_index()

In [953]:
# отобразим на графике
fig = make_subplots(rows=1, cols=1, shared_yaxes=False)

fig.add_trace(go.Bar(x = Office_table_count['area'], 
                     y = Office_table_count['count'],  
                     name='Полный день'),
              1, 1)

fig.add_trace(go.Bar(x = Gib_table_count['area'], 
                     y = Gib_table_count['count'],  
                     name='Гибкий график'),
              1, 1)

fig.add_trace(go.Bar(x = UD_table_count['area'], 
                     y = UD_table_count['count'],  
                     name='Удаленная работа'),
              1, 1)
    
fig.show()

In [ ]:
#Алексей, посмотрите пожалуйста, достаточно таких исследований ?
# выводы с графиков будут отображаться на слайдах
# надо ли прикручивать ML или этого будет досточно ?

In [ ]:
#все что ниже это всякое разное из разных моих домашних работ, наработки по ML и тд. Как все закончим, подчистим

In [687]:
#Перебираем все существующие требования к навыкам в поле key_skills и записываем их в лист
skill = []
for i in range(len(vac_info_new)): 
    if (vac_info_new['key_skills'][i]==None) != True: #смотрим только существующие значения
        x=vac_info_new['key_skills'][i]
        for j in range(len(x)): #и так как это словарь вытаскиваем значения ключа name
            skill.append(x[j])
#skill

In [689]:
# функцию нашел в инернетах, позволяет сравнением создать иной словарь
def analysis(your_list, your_dict):
    for i in your_list:
        if i in your_dict:
            your_dict[i] += 1
        else:
            your_dict[i] = 1

In [698]:
#создаем переменную под новый словарь и применяем функцию
dct_skill = {}
analysis(skill, dct_skill)
#dct_skill
#Сортируем по значениям а не ключам, и разворачиваем чтобы самое большое количество получить первым
dct_skill_sort = dict(sorted(dct_skill.items(), key=itemgetter(1), reverse=True))
#dct_skill_sort

In [701]:
#находим индексы максимальных значений, вдруг оно не одно 
index_for_max = []
for i in range(len(dct_skill_sort.values())):
    if list(dct_skill_sort.values())[i] == max(list(dct_skill_sort.values())):
        index_for_max.append(i)
    
#print(index_for_max)
    

[0]


In [705]:
# Выводим ключи максимальных значений
print('Самые востребованные навыки: ')
for i in index_for_max:
    print(list(dct_skill_sort.keys())[i])

Самые востребованные навыки: 
SQL


In [712]:
#Нагуглил круговую диаграмму и только через нее смог сделать:
#выводим топ 20 навыков в диаграмму

pyplt=py.offline.plot
labels=list(dct_skill.keys())[0:20]
values=list(dct_skill.values())[0:20]
trace=[go.Pie(labels=labels,values=values)]
layout=go.Layout(
    title='График процентов наиболее востребованных навыков'
)
fig=go.Figure(data=trace,layout=layout)
fig.show()

In [ ]:
#приведем поля name description key_skills к нижнему регистру
vac_info['name']=vac_info['name'].apply(lambda x: x.lower())

In [533]:
# def find_vasc (stroka):
#     """
#     функция поиска цифровых вакансий
#     """
#     vacs= ['аналитик данных', 'data scientist', 'data engineer', 'python', 'sql', 'java']
#     sovpalo = 0
#     for x in range(len(vacs)):
#         if (vacs[x] in stroka) == True:
#             sovpalo+=1
#         #else: return 0
#     if sovpalo > 0 : return 1
#     else: return 0

In [534]:
# #
# vac_info['flag'] = vac_info['name'].apply(find_vasc)


In [ ]:
# # функция удаления пунктуации и цифр
def remove_trash(list): 
    pattern = r'[^А-Яа-я]+'
    try:
      list = [re.sub(pattern, ' ', i) for i in list] 

    except Exception as e:
      print(e)
    return list

In [555]:
vac_info_new['description_clean']=remove_trash(vac_info_new['description'])

In [443]:
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [444]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Tolyan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [447]:
stop_russian = stopwords.words('russian')

In [450]:
morph = pymorphy2.MorphAnalyzer()

In [456]:
# функция лемматизации
def lemmatize(row):
    t = []
    text = row['description_clean']
    for word in text.split():
        if len(word)<=1:
            continue
        p = morph.parse(word)[0]
        t.append(p.normal_form)
    return " ".join(t)

In [556]:
%%time
#
vac_info_new['description_clean_normal'] = vac_info_new.apply(lemmatize,axis=1)


Wall time: 7.47 s


In [637]:
#vac_info_new.head(5)

In [558]:
text_transformer = TfidfVectorizer(stop_words=stop_russian, 
                                   ngram_range=(1,1), 
                                   lowercase=True)

In [559]:
vac_info_new_for_level = vac_info_new[['vac_level',  'professional_roles', 
                                                     'description_clean_normal']]

In [560]:
vac_info_new_for_level

,vac_level,professional_roles,description_clean_normal
0,None,"Программист, разработчик",наш команда объединять более специалист офис п...
1,Junior,"Программист, разработчик",корусый консалтинг один из крупный консалтинго...
2,Junior,Аналитик,адв дивизион где создаваться уникальный технол...
3,None,"Программист, разработчик",наш команда объединять более специалист офис п...
4,Junior,"Программист, разработчик",корусый консалтинг один из крупный консалтинго...
...,...,...,...
175,Junior,"Программист, разработчик",софтверный компания который уже более год явля...
176,Junior,"Программист, разработчик",софтверный компания который уже более год явля...
177,Junior,"Программист, разработчик",софтверный компания который уже более год явля...
178,Junior,"Программист, разработчик",софтверный компания который уже более год явля...


In [561]:
#уровень вакансий по тексту
leveled_vacs = vac_info_new_for_level[vac_info_new_for_level['vac_level'].notnull()].copy()
not_leveled_vacs = vac_info_new_for_level[vac_info_new_for_level['vac_level'].isnull()].copy()
leveled_vacs['vac_level'].unique()

array(['Junior'], dtype=object)

In [562]:
%%time
text_norm = text_transformer.fit_transform(leveled_vacs['description_clean_normal'])
X_train, X_test, y_train, y_test = train_test_split(text_norm, leveled_vacs['vac_level'], 
                                                    test_size=0.20, random_state=RANDOM_STATE)

Wall time: 26 ms


In [563]:
text_norm.shape

(120, 482)

In [511]:
leveled_vacs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 120 entries, 1 to 179
Data columns (total 3 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   vac_level                 120 non-null    object
 1   professional_roles        120 non-null    object
 2   description_clean_normal  120 non-null    object
dtypes: object(3)
memory usage: 3.8+ KB


In [512]:
not_leveled_vacs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60 entries, 0 to 158
Data columns (total 3 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   vac_level                 0 non-null      object
 1   professional_roles        60 non-null     object
 2   description_clean_normal  60 non-null     object
dtypes: object(3)
memory usage: 1.9+ KB


In [464]:
vac_info_new['vac_level']

0        None
1      Junior
2      Junior
3        None
4      Junior
        ...  
175    Junior
176    Junior
177    Junior
178    Junior
179    Junior
Name: vac_level, Length: 180, dtype: object

In [482]:
%%time


Wall time: 0 ns


<24x482 sparse matrix of type '<class 'numpy.float64'>'
	with 2552 stored elements in Compressed Sparse Row format>

In [564]:
%%time
clf = tree.DecisionTreeClassifier()
clf.fit(X_train,y_train)

Wall time: 2 ms


DecisionTreeClassifier()

In [565]:
%%time
pred = clf.predict(X_test)

Wall time: 1 ms


In [566]:
pred

array(['Junior', 'Junior', 'Junior', 'Junior', 'Junior', 'Junior',
       'Junior', 'Junior', 'Junior', 'Junior', 'Junior', 'Junior',
       'Junior', 'Junior', 'Junior', 'Junior', 'Junior', 'Junior',
       'Junior', 'Junior', 'Junior', 'Junior', 'Junior', 'Junior'],
      dtype=object)

In [516]:
print('Качество модели по метрике F1', f1_score(y_test,pred,average='weighted'))

Качество модели по метрике F1 1.0


In [517]:
text_norm_not_level = text_transformer.fit_transform(not_leveled_vacs['description_clean_normal'])

In [522]:
text_norm_not_level.shape

(60, 231)

In [519]:
pred_level=clf.predict(text_norm_not_level)

ValueError: X has 231 features, but DecisionTreeClassifier is expecting 482 features as input.

In [459]:
vac_info_new.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   id                        180 non-null    object        
 1   name                      180 non-null    object        
 2   area                      180 non-null    object        
 3   salary                    80 non-null     object        
 4   experience                180 non-null    object        
 5   schedule                  180 non-null    object        
 6   employment                180 non-null    object        
 7   description               180 non-null    object        
 8   key_skills                140 non-null    object        
 9   professional_roles        180 non-null    object        
 10  published_at              180 non-null    datetime64[ns]
 11  flag                      180 non-null    int64         
 12  salary_from           

In [469]:
vac_info_new.description[120]

'<p>Мы (компания &quot;Микролайн&quot; - www.microline.ru) разрабатываем приложения для управления ioT устройствами собственного производства (отопление, сигнализация, трекинг). Приложения у нас разные: веб, мобильные, десктопные. Объединяет их то, что мы стараемся разрабатывать их на веб-стеке: JS/TS, React, Redux. Нашими приборами и приложениями пользуются уже более 120 тысяч клиентов.</p> <p>Мы ищем веб-разработчика, который поможет нам создавать новые приложения и UI как для онлайн, так и для оффлайн использования.</p> <p><strong>Просим обратить особое внимание иногородних соискателей (!) Данная вакансия не предполагает удаленного режима работы (без каких-либо исключений).</strong></p> <p><strong>Разработчик, которого мы ищем, сможет:</strong></p> <ul> <li>Обсуждать бизнес-задачи, формулировать технические требования</li> <li>Придумывать способы реализации, обсуждать с командой пути решения, отстаивать свою точку зрения</li> <li>Разрабатывать классические клиент-серверные приложени

In [461]:
vac_info_new.description_clean[60]

' продуктовая компания мы созда м экосистему для брокерских компаний плагины под торговую платформу и систему копирования сделок Кого мы ищем разработчика готового работать в команде Что нужно будет делать Поддерживать и дополнять функционал Что жд м от кандидата Уверенное знание Уверенное знание и Опыт работы с Опыт работы с и умение писать запросы Английский на уровне чтения тех документации Готовность выполнить техническое задание Что предлагаем Слаженную но небольшую команду нас человек мы работаем из разных городов и стран Удал нную работу с гибким графиком Лояльное руководство отсутствие бюрократии отлаженный и комфортный процесс разработки Задачи любой сложности и работу над крутым продуктом Вакансия предполагает выполнение технического задания '

In [462]:
vac_info_new.description_clean_normal[60]

'продуктовый компания мы созд экосистема для брокерский компания плагин под торговый платформа система копирование сделка кто мы искать разработчик готовый работать команда что нужно быть делать поддерживать дополнять функционал что жд от кандидат уверенный знание уверенный знание опыт работа опыт работа умение писать запрос английский на уровень чтение тот документация готовность выполнить технический задание что предлагать слаженный но небольшой команда мы человек мы работать из разный город страна удалой нный работа гибкий график лояльный руководство отсутствие бюрократия отладить комфортный процесс разработка задача любой сложность работа над крутой продукт вакансия предполагать выполнение технический задание'

In [381]:
vac_info_new.professional_roles[64]

[{'id': '96', 'name': 'Программист, разработчик'}]